In [16]:
# import python libraries and nba library
import pandas as np
import nba_api as nba
from matplotlib import pyplot as plt
import pandas as pd
import socket
import logging

In [17]:
# import teams, players, and sort them into active or all players
from nba_api.stats.static import teams, players
nba_players = players.get_players()
active_players = players.get_active_players()
teams = teams.get_teams()

In [18]:
# import use-case specific functions
from nba_api.stats.endpoints import playercareerstats

In [19]:
# Add entry for each player on whether they have been analyzed or not. 
# Default is not analyzed
for player in active_players:
    player['Parsed ?'] = False
master_list = pd.DataFrame()

# Create temp dataframe and initialize count to serve as a staging 
# Dataframe and break up the merging into small batches
temp_df = pd.DataFrame()
count = 0

In [22]:
# Parse thru each active player
first_id = 0
last_id = 0
try:
    for player in active_players:

        # Check if they have been already analyzed
        if player['Parsed ?'] == True:
            pass
        
        # If they haven't, proceed
        else:
            passedId = player['id']
            player_name = player["full_name"]

            # Get player stats based on their ID
            career_stats = playercareerstats.PlayerCareerStats(player_id = passedId).get_data_frames()[0]
            
            # Get ID of first and last players analyzed for file naming purposes
            if first_id == 0:
                first_id = passedId
            last_id = passedId

            # Add player names to resulting dataframe to keep track of players
            career_stats.insert(1,"full_name",player_name)

            # Append resulting dataframe to temp dataframe
            frames = [temp_df,career_stats]
            temp_df = pd.concat(frames)

            # Set analyzed flag to true
            print(player_name)
            player['Parsed ?'] = True
            count = count + 1

            # Once enough volume is reached in the temp dataframe, append it to the main and re-initialize the dataframe
            if (count%20) == 0:
                frames = [master_list, temp_df]
                master_list = pd.concat(frames)
                temp_df = pd.DataFrame()
                print("Temp Full - Resetting")
except:
    print("TimeoutError")

TimeoutError


In [23]:
if first_id == 0:
    pass
else:
    master_list.to_csv("C:/Users/mario/OneDrive - University of Illinois - Urbana/Documents/Mario_External_HDD_Staging/Github Repo/Mario-Jupyter-Codespace/data/basketballProject/nba_data_"+str(first_id)+"to"+str(last_id)+".csv")